## Tensorflow/Keras Tutorial

In [ ]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

### Sequential networks; layers
- A **feed-forward** or **sequential** neural network is constructed using the `tf.keras.Sequential` class construtor.
It takes a list of **layers** as input. The last layer in the list is the output layer. The rest are hidden layers.
- Layers are defined in the `tf.layers.keras` module.
- In a **dense** layer, a unit is connected with every unit in the previous layer.
- The first layer in a sequential network is responsible for specifying the dimension of the data, i.e., the number of columns of the data matrix, in the `input_dim` keyword argument.
- A sequential model stores its list of layers in its `layers` attribute.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_dim=2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
print(f"Layers: {model.layers}")
model.summary()

### Weights
- Weights associated to connections between units are initialized randomly. Biases are initialized to zero.
- Inspect the weights using the `get_weights` method.
- It returns a list of length $2L$, $L$ being the number of layers, containing the weight matrices and bias vectors of each layer.
- The weight matrix of dense layer $\ell$ has size $p_{\ell - 1}\times p_\ell$, where $p_\ell$ is the number of units in the $\ell$-th layer.

In [ ]:
w1, b1, w2, b2 = model.get_weights()
model.set_weights([np.zeros_like(w1), b1, np.zeros_like(w2), b2])
print(model.get_weights())
model.set_weights([w1, np.ones_like(b1), w2, np.ones_like(b2)])

### Forward propagation; prediction
- If $W^{[\ell]}$, $b^{[\ell]}$, and $A_\ell$ are the weight matrix, and bias vector and activation function of layer $\ell$ and $a^{[\ell]}$ and $a^{[\ell-1]}$ is the vector of activations of layers $\ell$ and $\ell - 1$, then $$a^{[\ell]} = A_\ell(a^{[\ell - 1]}W^{[\ell]} + b^{[\ell]}).$$
- We define the activations of the input layer to be the input values, themselves.
- To propagate data through a network, use the `predict` method.
- To check our understanding, we can do this manually.

In [ ]:
X = np.random.normal(size=(10, 2))
print(model.predict(X))

relu = tf.keras.activations.relu
sigmoid = tf.keras.activations.sigmoid

w1, b1, w2, b2 = model.get_weights()
a0 = X
a1 = relu(a0.dot(w1) + b1).numpy()
a2 = sigmoid(a1.dot(w2) + b2)

print(a2)

### Exercise

- Encode this 1-layer neural network with sigmoid activation as a Keras sequential model. Set the weights and bias as indicated.
- Predict the values for $(0, 0)$, $(1, 0)$,  $(0, 1)$, and $(1, 1)$.

<img src="one_and_three_nn.png" height="50%" width="50%" />

### Exercise

- Encode this 2-layer neural network as a Keras sequential model. Set the weights and bias as indicated.
- Predict the values for $(0, 0)$, $(1, 0)$,  $(0, 1)$, and $(1, 1)$.

<img src="xnor.png" height="50%" width="50%" />

### Training

- In practice we don't assign weights; the network **learns** the weights from **training data**.
- Given a data point $(\boldsymbol{x}, y)$, can feed $\boldsymbol{x}$ into the network which returns a quantity $\widehat y$.
- Error in approimating of $y$ by $\widehat y$ is encoded in a **loss function** $L(y, \widehat{y})$.
- Which loss function to use is part of the design of your machine learning algorithm. It is not intrinsic to the neural network; you must provide it to the `compile` method.
- By **backpropagating** this error through the network, `tensorflow` systematically adjusts the weights and biases to reduce the error.
Specity the **optimizer**, the numerical algorithm used to adjust the weights and biases, as the `optimizer` argument of the model's `compile` method.
- A training loop can makes multiple passes over the training date. Each pass through an **epoch**. Specify the number of training epochs to the `compile` method as the `epochs` argument.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_dim=2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

X = np.random.normal(size=(1, 2))
y = np.random.randint(2, size=(1,))
print(X, y, "\n")

y_p = model.predict(X)

loss_should_be = tf.keras.losses.binary_crossentropy(y, y_p)
print(f"The cross entropy loss should be: {loss_should_be}\n")

h = model.fit(X, y)

In [ ]:
X = np.random.normal(size=(20, 2))
y = np.random.randint(2, size=(20,))
expected_training_accuracy = np.sum(model.predict_classes(X).reshape(-1) == y)/len(y)
print(f"Expected training accuracy: {expected_training_accuracy}\n")
model.fit(X, y)

In [ ]:
n_train = 10000
n_test = 1000
X_train = np.random.randint(2, size=(n_train, 2))
X_test = np.random.randint(2, size=(n_test, 2))
y_train = np.logical_not(np.logical_xor(X_train[:, 0], X_train[:, 1]))*1
y_test = np.logical_not(np.logical_xor(X_test[:, 0], X_test[:, 1]))*1
X_train = X_train + np.random.normal(0, 0.2, size=(n_train, 2))
X_test = X_test + np.random.normal(0, 0.2, size=(n_test, 2))
plt.plot(X_train[y_train == 0, 0], X_train[y_train == 0, 1], 'ro')
plt.plot(X_train[y_train == 1, 0], X_train[y_train == 1, 1], 'bo')
plt.show()

model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_dim=2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

In [ ]:
y_p = model.predict_classes(X_test).reshape(-1)
sum(y_p == y_test)/len(y_p)

In [ ]:
U = np.random.uniform(size=(1000, 2))
v = model.predict_classes(U).reshape(-1)
plt.plot(U[v == 0, 0], U[v == 0, 1], 'ro')
plt.plot(U[v == 1, 0], U[v == 1, 1], 'bo')
plt.show()


### Exercise
Build a network that can recognize this pattern

In [ ]:
plt.plot(range(30), h["val_loss"], range(30), h["loss"], range(30),  h["val_accuracy"], range(30), h["accuracy"])
plt.show()

### Exercise

- Construct a sequential neural network that can learn the pattern depicted below.
- Experiment with numbers of layers, numbers of units in each layer, optimizer (`optimizer="rmsprop"`) typically works well), etc.
- Note any interesting misclassification patterns you find. 

<img src="boxed_circle.png" />


In [ ]:
# Here's how to generate data like this:
n = 2**10
X = np.random.uniform(size=(n, 2))
y = np.linalg.norm(X - np.array([0.5, 0.5]), axis=1) < 0.4